**Scenario:**

You are a junior data analyst working in a bike-share company in Chicago. your team wants to understand how casual riders and annual members use Cyclistic bikes differently.

The marketing analyst team needs to better understand

1. how annual members and casual riders differ,
2. why casual riders would buy a membership, and
3. how digital media could affect their marketing tactics.

The director of marketing has assigned you the first question to answer: 
> How do annual members and casual riders use Cyclistic bikes differently?

# Case Study Roadmap - Ask

**Guiding questions**

1. What is the problem you are trying to solve? <br>
To find how casual riders and anual members use the services differently.

2. How can your insights drive business decisions? <br>
Insights will show how each rider type use the services differently, if there will be a benefit converting the casual riders to annual members.

**Key tasks**

Identify the business task: <br>
Find how casual riders use the service differently from the annual members. (Initial thoughts on what may be different: patterns, reasons, duration, distance, time of the day, day of the week, month of the year, locations.)

Consider key stakeholders: <br>
Analytics team, marketing team, director of marketing.

**Deliverable:**

A clear statement of the business task: <br>

> Report showing how a casual rider uses the bikes differently than an annual member.

# Case Study Roadmap - Prepare

**Guiding questions**

1. Where is your data located? <br>
Cyclistic’s historical trip data. [Download 12 months of Cyclistic trip data here.](https://divvy-tripdata.s3.amazonaws.com/index.html) 

2. How is the data organized? <br> Data is organized in monthly trip data based on start date. Each month's data is in different csv file. I used 12 months of data from 2022 January to 2022 December.

3. Are there issues with bias or credibility in this data? Does your data ROCCC? <br>
Data is provided free with a limited license. It doesn't have any personal information. ROCCC assumed (Reliable, Original, Comprehensive, Current, Cited)

4. How are you addressing licensing, privacy, security, and accessibility? <br>
(The data has been made available by Motivate International Inc. under this [licence](https://ride.divvybikes.com/data-license-agreement)

5. How did you verify the data’s integrity? <br> Assumed.
6. How does it help you answer your question? <br> Analyze phase.
7. Are there any problems with the data? <br> 
    1. September data name is different from others.

# Case Study Roadmap - Process

**Guiding questions**

1. What tools are you choosing and why? <br>
    Initially I chose google sheets. I started using a single month. Even then the operations were slow. But it helped me to see some issues visually. Such as: missing values in start station, end station. Very long ride lengths, sometimes over a week.
    Then I switched to R.
2. Have you ensured your data’s integrity?
3. What steps have you taken to ensure that your data is clean? Assumed
4. How can you verify that your data is clean and ready to analyze? Documented below
5. Have you documented your cleaning process so you can review and share those results? Documented below

**Key tasks**

Deliverable Documentation of any cleaning or manipulation of data:

- Downloaded last 12 months data from the link. 2022 February till 2023 January. Uploaded this to my google drive to use with google sheets. It takes 40 seconds to upload.
- Take 2023 January File to work on initially. To open a file initially takes 60 seconds as it is converted from cvs to google sheets format.<
- There are 13 columns: <br>ride_id: 16 digit hex unique ride id (65DBD2F447EC51C2) <br>rideable_type: 3 choices (classic_bike, docked_bike, electric_bike) <br>started_at: date and time (12/5/2022 10:47:18) <br>ended_at: date and time (12/5/2022 10:56:34) <br>start_station_name: intersection of 2 streets (Clifton Ave & Armitage Ave) <br>start_station_id: unique? station id, 2 different formats (TA1307000163) or (13277) <br>end_station_name: intersection of 2 streets (Clifton Ave & Armitage Ave) <br>end_station_id: unique? station id, 2 different formats (TA1307000163) or (13277) <br>start_lat: (41.9182435) <br>start_lng: (-87.65711467) <br>end_lat: (41.922167) <br>end_lng: (-87.638888) <br>member_casual: 2 choices (member, casual)<br>
-Select columns started_at & ended_at. Change the format to DateTime (Format->Number->Date Time)<br>
-Add a new column (ride_length) change the format to duration (Format->Number->Duration)<br>
-Enter a formula and paste to the whole column, =(D2-C2)<br>
-Added a new column (day_of_week), used formula =Weekday(c2) and pasted the whole column.<br>
-Changed the format of day_of_week to number (initially it looked like dateTime) (Format->Number->0)<br>
-Added 2 more columns start_month, end_month, put formula = Month(c2), and change the format to number.<br>
-Final 16 columns: <br>ride_id, rideable_type, day_of_week, started_at, start_month, ended_at, end_month, ride_length, start_station_name, start_station_id, end_station_name, end_station_id, start_lat, start_lng, end_lat, end_lng,member_casual

# Case Study Roadmap - Process -2 
# 
Guiding questions

1. What tools are you choosing and why? <br>I switched to R Studio as google sheets can not handle 12 files, around 6 million records.


---------------------------------------------------------------------------------------

**Before going into analysis by R, I want to thank and acknowledge the study on the same subject by JHELISON GABRIEL LIMA UCHOA. I copied and studied his R scripts to learn. I also used some of his scripts in this study.** [Here is a link to his case study](https://www.kaggle.com/code/jhelisonuchoa/google-data-analytics-capstone-case-study-1)


---------------------------------------------------------------------------------------

In [ ]:
#I want to use 2022 data from January to December
#This location also includes January of 2023 which I don't want to include in my analysis
#create a list of file names in this data path
dataFiles <- list.files(path = "/kaggle/input/google-casestudy-1-2022-2023", include.dirs=TRUE)
#remove 2023 data
dataFiles <- dataFiles[dataFiles != "202301-divvy-tripdata.csv"]
#add data path to file name
dataFiles <- paste("/kaggle/input/google-casestudy-1-2022-2023",dataFiles, sep="/")


library(data.table)
#read all these files and merge them into a big data frame
rideData = do.call(rbind, lapply(dataFiles, fread))


In [ ]:
#if you experience problems subtracting ended_at - started_at
#try following lines
#>rideData$started_at <- as.POSIXct(rideData$started_at, "%Y-%m-%d %H:%M:%S", tz="")
#>rideData$ended_at <- as.POSIXct(rideData$ended_at, "%Y-%m-%d %H:%M:%S", tz="")

#adding a length of ride field in seconds
library(dplyr)
rideData <- rideData %>%
  mutate(ride_length = as.numeric(rideData$ended_at - rideData$started_at))

#looking at the data
library(skimr)
skim_without_charts(rideData)

# Data Quality:
1. There are 833064 missing start_station_name and start_station_id.
2. There are 892742 missing end_station_name and end_station_id.
3. There are only 5858 missing end_lat and end_lng. If stations have unique lat and lng, then missing station_name and station_id values may be mostly recovered from lat and lng. **Should be checked.**
4. ride_length has negative values and very big values. These are clearly not right.<br>
    ride_length < 0 : clearly an IT issue, started_at is later than ended_at. These data should be removed<br>
    ride_length < 60 sec : people may just canceled the ride because of any reason. 60 second is picked, but it could be 30 seconds or 90 seconds. Probably start_station is same as the end_station for these records. **Should be checked.**<br>
    ride_length > 10,800 sec (3 hours) : people may have not returned the bike to the bike station. In a hurry they may just left the bike where ever was convenient. Maybe they were trying to catch a taxi for an appointment, for a plane ride. Whatever the reason they probably did not return the bike. **Should be checked.**<br>

In [ ]:
#Before cleaning up the data let's get some statistical values of the raw data
#to make a comparison after cleaning the data

#Let's add a day_of_week column to our data so we can make statistical evaluations
#depending on the day of the week
#Sun=1, Mon=2, Tue=3, Wed=4, Thu=5, Fri=6, Sat=7
rideData$day_of_week <- wday(rideData$started_at)

#Want to find the average ride length in minutes, grouped by member type
rideData %>%
  summarize(mean_ride_length_in_min = mean(ride_length/60))

#Want to find the average ride length in minutes, grouped by member type
rideData %>%
  group_by(member_casual) %>%
  summarize(mean_ride_length_in_min = mean(ride_length/60))

#Want to find the average ride length in minutes, grouped by member type and day of the week
rideData %>%
  group_by(member_casual, day_of_week) %>%
  summarize(mean_ride_length_in_min = mean(ride_length/60))

#R doesn't have mode functionality so we use modeest package and use mfv function
# Import the library
install.packages("modeest")
library(modeest)

#We find the day of the week members use the service the most
rideData %>%
  group_by(member_casual) %>%
  summarize(mode_day_of_week = mfv(day_of_week))
#For casual riders it is Saturday
#For members it is Thursday

In [ ]:
#Let's start removing the records for ride lengths
#Less than 0 seconds  : An error
#Less than 60 seconds : A canceled ride
#More than 10800 seconds (3 hours) : Bike is not returned back to a station, abondoned.
#This is still a ride but skews the metrics

#Rides with less than 0 duration - remove
#move less than 0 ride_length to lessThanZeroRide
#if we want to analyze more those rides, for now they are commented
#lessThanZeroRide <- rideData[rideData$ride_length < 0]
rideData <- subset(rideData, rideData$ride_length >= 0)

#Rides that are immediately canceled - remove
#closeToZeroRide <- rideData[rideData$ride_length < 60]
rideData <- subset(rideData, rideData$ride_length >= 60)

#Rides longer than 3 hours remove
#veryLongRide <- rideData[rideData$ride_length > 10800]
rideData <- subset(rideData, rideData$ride_length <= 10800)

In [ ]:
#Perform the same calculations as before

#Want to find the average ride length in minutes, grouped by member type
rideData %>%
  summarize(mean_ride_length_in_min = mean(ride_length/60))

#Want to find the average ride length in minutes, grouped by member type
rideData %>%
  group_by(member_casual) %>%
  summarize(mean_ride_length_in_min = mean(ride_length/60))

#Want to find the average ride length in minutes, grouped by member type and day of the week
rideData %>%
  group_by(member_casual, day_of_week) %>%
  summarize(mean_ride_length_in_min = mean(ride_length/60))

#We find the day of the week members use the service the most
rideData %>%
  group_by(member_casual) %>%
  summarize(mode_day_of_week = mfv(day_of_week))
#For casual riders it is Saturday
#For members it is Thursday

Here are the results.

|Before Cleanup|After cleanup|
|---|---|
|mean_ride_length (min)|mean_ride_length (min)|
|19.44359|15.63968|


                                Before Cleanup          After Cleanup
|member_casual|avg ride min|count|%|avg ride min|count|%|
|-------|-------|---|---|-------|---|---|
|casual|29.1|2322032|41.0|20.5|2258559|40.9|
|member|12.7|3345685|59.0|12.3|3268475|59.1|

|member_casual |day_of_week |mean_ride_length (min) | count | %|mean_ride_length (min) | count | %|
|---|---|---|---|---|---|---|---|
|casual        |1|34.1|389036|6.86|23.3|377718|6.83|
|casual        |2|29.2|277675|4.90|20.9|270080|4.89|
|casual        |3|25.8|263746|4.65|18.4|256762|4.65|
|casual        |4|24.8|274354|4.84|17.9|267330|4.84|
|casual        |5|25.5|309330|5.46|18.4|301258|5.45|
|casual        |6|28.0|334701|5.91|19.3|325657|5.89|
|casual        |7|32.6|473190|8.35|23.0|459754|8.32|
|member        |1|14.0|387223|6.83|13.5|377671|6.83|
|member        |2|12.3|473339|8.35|11.8|462715|8.37|
|member        |3|12.1|518626|9.15|11.6|507204|9.18|
|member        |4|12.1|523869|9.24|11.7|512120|9.27|
|member        |5|12.3|532261|9.39|11.9|520170|9.41|
|member        |6|12.5|467086|8.24|12.1|456332|8.26|
|member        |7|14.1|443281|7.82|13.6|432263|7.82|

----------------------------------------------------------------------

As you can see, after cleaning up the data, average ride lengths decrease quite a lot for casual users. Member average length also drops but it is a small decrease.

----------------------------------------------------------------------

At this point, I am not sure if 3 hours is a good cut for removing long rides. [Divvy Bikes Pricing](https://divvybikes.com/pricing) allows a day pass to drive for 3 hours for 16.50 dollars. Some people may just continue riding for some more time for 0.17 cents a minute after 3 hours. Maybe 180 minutes * 1.50 is a better cutoff. Extra 1.5 hours cost 15.30 dollars. Anyways, I will just go with 3 hours cut off. (I cannot put dollar sign because the formatting of this text changes)

Still working on cleaning data. I want to see how starting_station_name (and id), end_station_name ( and id) affects the data. I also want to see the most common routes. So I add a new column called ride_route.

In [ ]:
#Now I want to tackle the empty station id data
#Add a route column with start_station_name -> end_station_name
#df$x <- paste(df$n, "-", df$s)
rideData$ride_route <- paste(rideData$start_station_name, " -> ", rideData$end_station_name)

#sort according to the frequency of the route while filtering for member_casual
library(dplyr)

rideData %>% 
  group_by(ride_route) %>%
  filter(member_casual == "member") %>% 
  summarize(count = length(ride_route)) %>% 
  arrange(desc(count))

rideData %>% 
  group_by(ride_route) %>%
  filter(member_casual == "casual") %>% 
  summarize(count = length(ride_route)) %>% 
  arrange(desc(count))

Results for member:<br>


|ride_route|                                           count|
|----|----|
|   <chr>     |                                           <int>|
| 1 "  ->  "  |                                          220743|
| 2 "Ellis Ave & 60th St  ->  University Ave & 57th St" |  5844|
| 3 "University Ave & 57th St  ->  Ellis Ave & 60th St" |  5537|
| 4 "Ellis Ave & 60th St  ->  Ellis Ave & 55th St"      |  5276|
| 5 "Ellis Ave & 55th St  ->  Ellis Ave & 60th St"      |  4743|
| 6 "State St & 33rd St  ->  Calumet Ave & 33rd St"     |  3264|
| 7 "Calumet Ave & 33rd St  ->  State St & 33rd St"     |  3215|
| 8 "Loomis St & Lexington St  ->  Morgan St & Polk St" |  2891|
| 9 "Morgan St & Polk St  ->  Loomis St & Lexington St" |  2876|
|10 "University Ave & 57th St  ->  "                    |  2287|

<br>
Results for casual: <br>
    
|   ride_route   |                                                               count|
|---|---|
|   <chr>                          |                                             <int>|
| 1 "  ->  "                        |                                           177227|
| 2 "Streeter Dr & Grand Ave  ->  Streeter Dr & Grand Ave"                     |  9592|
| 3 "DuSable Lake Shore Dr & Monroe St  ->  DuSable Lake Shore Dr & Monroe St" |  6080|
| 4 "DuSable Lake Shore Dr & Monroe St  ->  Streeter Dr & Grand Ave"           |  5079|
| 5 "Michigan Ave & Oak St  ->  Michigan Ave & Oak St"                         |  4213|
| 6 "Millennium Park  ->  Millennium Park"                                     |  3635|
| 7 "Streeter Dr & Grand Ave  ->  DuSable Lake Shore Dr & Monroe St"           |  2845|
| 8 "Streeter Dr & Grand Ave  ->  "                                            |  2749|
| 9 "Montrose Harbor  ->  Montrose Harbor"                                    |   2729|
|10 "Streeter Dr & Grand Ave  ->  Millennium Park"                            |   2725|    

 
 As you can see, empty station name is a huge problem if we want to know about the different routes riders use, depending on being a member or casual.

Ignoring the blank routes for now (which is a big problem), do you see the beauty of the routes. It eliminates the accumulation of the bikes in one location. <br>
For members, top 10 routes<br>
route 1 is empty string <br>
route 2 is reverse of route 3<br>
route 4 is reverse of route 5<br>
route 6 is reverse of route 7<br>
route 8 is reverse of route 9<br>
route 10, end point is empty<br>

For casuals, top 10 routes<br>
route 1 is empty string<br>
route 2 is a loop<br>
route 3 is a loop<br>
route 4 is not a loop<br>
route 5 is a loop<br>
route 6 is a loop<br>
route 7 is not a loop<br>
route 8, end point is empty<br>
route 9 is a loop<br>
route 10 is not a loop<br>